In [ ]:
# generating sorted lists of NCBI numbers and sequences from gene.json for protein and nucleotide
import pandas as pd
import json
import re

with open('gene.json') as f:
    raw_data = json.load(f)
try:
      # normalizing data (without "success")
      data_normalized = pd.json_normalize(raw_data['success'])
except Exception as e:
      print(f"An error occurred during normalization: {e}")

df_ncbi = data_normalized[["gene_ncbi","sequence"]]
display(df_ncbi)
#6754 rows

# 1. filter out without NCBI number and cleaning GI number format
pattern = r'^GI:[A-Za-z0-9]+$'
mask_invalid_format = ~df_ncbi['gene_ncbi'].str.match(pattern, na=False)
df_valid = df_ncbi[~mask_invalid_format].copy()
display(df_valid)
#6752 rows
df_valid['gene_ncbi'] = df_valid['gene_ncbi'].str.replace('^GI:', '', regex=True)
df_valid['sequence'] = df_valid['sequence'].str.upper()
display(df_valid)

# 2. without sequence, split into p and n

mask_na_or_empty = df_valid['sequence'].isna() | (df_valid['sequence'].str.strip() == '')
df_valid_no_seq = df_valid[mask_na_or_empty].copy()
display(df_valid_no_seq)
#434

# Split based on the last letter
df_valid_no_seq['last_char'] = df_valid_no_seq['gene_ncbi'].str[-1]

df_p_no_seq = df_valid_no_seq[df_valid_no_seq['last_char'] == 'p'].drop(columns='last_char')
df_p_no_seq = df_p_no_seq['gene_ncbi']
df_p_no_seq = df_p_no_seq.map(lambda x: ''.join(filter(str.isdigit, str(x))))
display(df_p_no_seq)
#366 rows
# to file
df_p_no_seq.to_csv("df_p_no_seq.csv", index=False)

df_no_letter_no_seq = df_valid_no_seq[~df_valid_no_seq['last_char'].isin(['p', 'n'])].drop(columns='last_char')
df_no_letter_no_seq = df_no_letter_no_seq['gene_ncbi']
display(df_no_letter_no_seq)
#68 rows
# to file
df_no_letter_no_seq.to_csv("df_no_letter_no_seq.csv", index=False)
# no_letter changed to n in the local file name

# 3. with sequence to fasta file, split into p and n

mask_na_or_empty = df_valid['sequence'].isna() | (df_valid['sequence'].str.strip() == '')
df_valid_seq = df_valid[~mask_na_or_empty].copy()
display(df_valid_seq)
#6318 rows

df_valid_seq['last_char'] = df_valid_seq['gene_ncbi'].str[-1]

df_p = df_valid_seq[df_valid_seq['last_char'] == 'p'].drop(columns='last_char')
df_p['gene_ncbi'] = df_p['gene_ncbi'].map(lambda x: ''.join(filter(str.isdigit, str(x))))
display(df_p)
#2342 rows
# to file
df_p.to_csv("df_p.csv", index=False)

df_no_letter = df_valid_seq[~df_valid_seq['last_char'].isin(['p', 'n'])].drop(columns='last_char')
display(df_no_letter)
#3976 rows
# to file
df_no_letter.to_csv("df_no_letter.csv", index=False)
# no_letter changed to n in the local file name

In [ ]:
# analysis of information from E-Direct, success of obtaining accession number, duplicates between protein, nucleotide, gene, remaining records for searching

import pandas as pd
import re

# full_list protein vs full_list nucleotide vs full_list genes
# repeat for full_list and no_seq_list

df_p = pd.read_csv('output_GI_to_accesion_protein_emailversion.tsv', sep='\t')
df_n = pd.read_csv('output_GI_to_accesion_nucleotide_emailversion.tsv', sep='\t')
df_g = pd.read_csv('output_GI_to_accesion_gene_emailversion.tsv', sep='\t')
#pd.DataFrame(columns=["GI", "AccessionVersion"])

#display(df_p)
#display(df_n)
#display(df_g)


# what part of protein was mapped
missing_count = df_p['AccessionVersion'].isna().sum() + (df_p['AccessionVersion'].astype(str).str.strip() == '').sum()
mask_na_or_empty = df_p['AccessionVersion'].isna() | (df_p['AccessionVersion'].str.strip() == 'nan')
df_p_filtered = df_p[~mask_na_or_empty].copy()
df_p_filtered.to_csv("df_p_filtered.csv", index=False)
print("Protein DB")
print(f"Number of empty cells: {missing_count}")
print(f"Total number of rows: {len(df_p)}")
print(f"Number of retrieved accession numbers: {len(df_p_filtered)}")

# what part of nucleotide was mapped
missing_count = df_n['AccessionVersion'].isna().sum() + (df_n['AccessionVersion'].astype(str).str.strip() == '').sum()
mask_na_or_empty = df_n['AccessionVersion'].isna() | (df_n['AccessionVersion'].str.strip() == 'nan')
df_n_filtered = df_n[~mask_na_or_empty].copy()
df_n_filtered.to_csv("df_n_filtered.csv", index=False)
print("Nucleotide DB")
print(f"Number of empty cells: {missing_count}")
print(f"Total number of rows: {len(df_n)}")
print(f"Number of retrieved accession numbers: {len(df_n_filtered)}")

# what part of gene was mapped
missing_count = df_g['AccessionVersion'].isna().sum() + (df_g['AccessionVersion'].astype(str).str.strip() == '').sum()
mask_na_or_empty = df_g['AccessionVersion'].isna() | (df_g['AccessionVersion'].astype(str).str.strip() == 'nan')
df_g_filtered = df_g[~mask_na_or_empty].copy()
df_g_filtered.to_csv("df_g_filtered.csv", index=False)
print("Gene DB")
print(f"Number of empty cells: {missing_count}")
print(f"Total number of rows: {len(df_g)}")
print(f"Number of retrieved accession numbers: {len(df_g_filtered)}")



# Compare columns between protein, nucleotide, genes accession mapping results = are there duplicate mappings
setp = set(df_p_filtered['GI'].dropna().astype(str).str.strip())
setn = set(df_n_filtered['GI'].dropna().astype(str).str.strip())
setg = set(df_g_filtered['GI'].dropna().astype(str).str.strip())


common_pn = setp & setn
common_pg = setp & setg
common_ng = setn & setg

pn_only_p = setp - setn
pn_only_n = setn - setp
pg_only_p = setp - setg
pg_only_g = setg - setp
ng_only_n = setn - setg
ng_only_g = setg - setn


print(f"Common values pn: {len(common_pn)}")
print(f"Common values pg: {len(common_pg)}")
print(f"Common values ng: {len(common_ng)}")

print(f"PN only in p: {len(pn_only_p)}")
print(f"PN only in n: {len(pn_only_n)}")
print(f"PG only in p: {len(pg_only_p)}")
print(f"PG only in g: {len(pg_only_g)}")
print(f"NG only in n: {len(ng_only_n)}")
print(f"NG only in g: {len(ng_only_g)}")

In [ ]:
import pandas as pd
import re

# lists of p and n AccessionVersion to download fasta from NCBI (filter out those for which I have sequences) + missing record

# df_p_filtered and df_n_filtered from the previous block are two-column dfs with GI and AccessionVersion
df_p_filtered = pd.read_csv('df_p_filtered.csv')
df_n_filtered = pd.read_csv('df_n_filtered.csv')
df_p_clean_list = pd.read_csv('df_p.csv') # columns gene_ncbi,sequence
df_p_no_seq_clean_list = pd.read_csv('df_p_no_seq.csv') # column gene_ncbi
df_n_clean_list = pd.read_csv('df_n.csv') # columns gene_ncbi,sequence
df_n_no_seq_clean_list = pd.read_csv('df_n_no_seq.csv') # column gene_ncbi

# unifying column names

df_p_clean_list = df_p_clean_list.rename(columns={'gene_ncbi': 'GI'})
df_p_no_seq_clean_list = df_p_no_seq_clean_list.rename(columns={'gene_ncbi': 'GI'})
df_n_clean_list = df_n_clean_list.rename(columns={'gene_ncbi': 'GI'})
df_n_no_seq_clean_list = df_n_no_seq_clean_list.rename(columns={'gene_ncbi': 'GI'})

# adding AccessionNumber = merge to a 3-column df

df_p_clean_list = pd.merge(df_p_clean_list, df_p_filtered, on='GI', how='left')
df_p_no_seq_clean_list = pd.merge(df_p_no_seq_clean_list, df_p_filtered, on='GI', how='left')
df_n_clean_list = pd.merge(df_n_clean_list, df_n_filtered, on='GI', how='left')
df_n_no_seq_clean_list = pd.merge(df_n_no_seq_clean_list, df_n_filtered, on='GI', how='left')

display(df_n_filtered)
display(df_p_clean_list)
display(df_p_no_seq_clean_list)
display(df_n_clean_list)
display(df_n_no_seq_clean_list)

# extracting those NaNs that were supposed to be p but are n (df_n_clean_list)

mask_na_or_empty = df_n_clean_list['AccessionVersion'].isna() | (df_n_clean_list['AccessionVersion'].astype(str).str.strip() == 'nan')
df_n_nan = df_n_clean_list[mask_na_or_empty].copy()
#display(df_n_nan)
df_n_nan = df_n_nan.drop(columns=['AccessionVersion'])
df_n_nan = pd.merge(df_n_nan, df_p_filtered, on='GI', how='left')
df_n_nan['NCBI_DB'] = df_n_nan['AccessionVersion'].notna().map({True: 'P', False: ''})
display(df_n_nan)

df_n_clean_list = df_n_clean_list[~mask_na_or_empty]
display(df_n_clean_list)
df_n_clean_list['NCBI_DB'] = df_n_clean_list['AccessionVersion'].notna().map({True: 'N', False: ''})
display(df_n_clean_list)

df_n_clean_list_ncbi_db = pd.concat([df_n_clean_list, df_n_nan], ignore_index=True)
display(df_n_clean_list_ncbi_db)

# clean lists of AccessionVersion to download KO

df_KO_p_all = pd.concat([df_p_clean_list['AccessionVersion'], df_p_no_seq_clean_list['AccessionVersion'], df_n_nan['AccessionVersion']], ignore_index=True)
display(df_KO_p_all)

df_KO_n_all = pd.concat([df_n_clean_list['AccessionVersion'], df_n_no_seq_clean_list['AccessionVersion']], ignore_index=True)
display(df_KO_n_all)

df_KO_p_all.to_csv("df_KO_p_all.csv", index=False)
df_KO_n_all.to_csv("df_KO_n_all.csv", index=False)

# df for generating fasta directly from file
# df_p_clean_list
df_p_clean_list['sequence'] = df_p_clean_list['sequence'].str.replace(r'\s+', '', regex=True)
df_p_clean_list.to_csv("df_p_clean_list.csv", index=False)
# df_n_clean_list
df_n_clean_list_ncbi_db['sequence'] = df_n_clean_list_ncbi_db['sequence'].str.replace(r'\s+', '', regex=True)
df_n_clean_list_ncbi_db.to_csv("df_n_clean_list_ncbi_db.csv", index=False)

In [ ]:
# script for generating FASTA from sequences in BioSurfDB (+AccessionNumber)
df_p_clean_list = pd.read_csv('df_p_clean_list.csv')
df_n_clean_list_ncbi_db = pd.read_csv('df_n_clean_list_ncbi_db.csv')

# generating fasta for df_p_clean_list
with open('output_p.fasta', 'w') as fasta_file:
    for _, row in df_p_clean_list.iterrows():
        id_part = f"{str(row['GI']).strip()},{str(row['AccessionVersion']).strip()}"
        sequence = str(row['sequence']).strip()

        if id_part and sequence:
            fasta_file.write(f">{id_part}\n{sequence}\n")

# generating fasta for df_n_clean_list_ncbi_db
with open('output_n.fasta', 'w') as fasta_file:
    for _, row in df_n_clean_list_ncbi_db.iterrows():
        id_part = f"{str(row['GI']).strip()},{str(row['AccessionVersion']).strip()},{str(row['NCBI_DB']).strip()}"
        sequence = str(row['sequence']).strip()

        if id_part and sequence:
            fasta_file.write(f">{id_part}\n{sequence}\n")